<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2023/ex15notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2023 ex15notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo15.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の DCT と FFT の関数
from scipy.fft import dct, fft, ifft, fftfreq

# 科学技術計算のライブラリ SciPy の中の WAVE ファイルを扱うモジュール
from scipy.io.wavfile import read, write

----
## 演習問題 - 音のデータの DFT
----

音のデータをDFTして分析してみましょう．

---
### 準備

ネットから WAVE 形式のファイルを入手するので，WAVE形式のファイルを読み込む関数を定義しておきます．

In [ ]:
# WAVE ファイルを読み込む関数
#
def readWAVE(filename):

    framerate, data = read(filename)

    if data.ndim == 1:
        nchannels = 1
    else:
        nchannels = data.shape[1]
    nframes = data.shape[0]

    print('filename = ', filename)
    print('nchannels = ', nchannels)       # チャンネル数
    print('framerate = ', framerate)       # 標本化周波数
    print('nframes = ', nframes)             # フレーム数
    print('duration = {0:.2f}[sec]'.format(nframes / framerate))   # 長さ（秒単位）
    print('dtype = ', data.dtype)            # データ型（量子化ビット数に対応）

    assert data.dtype == 'uint8' or data.dtype == 'int16' or data.dtype == 'int32' or data.dtype == 'float32'

    # 最初の10秒分だけ取り出す（元がそれより短ければそのまま）
    nframesNew = min(framerate * 10, nframes)
    if nchannels == 1:
        dataNew = data[:nframesNew]
    else:
        # 多チャンネル信号なら0番目と1番目の平均値を取り出す
        if data.dtype == 'float32':  # 浮動小数点数のときは [0, 1] の値なので普通に足して2で割る
            dataNew = (data[:nframesNew, 0] + data[:nframesNew, 1])/2
        else: # 整数型のときはオーバーフローする可能性があるので，いったん64bit整数にしてから
            data64 = (data[:nframesNew, 0].astype(np.int64) + data[:nframesNew, 1].astype(np.int64))//2
            dataNew = data64.astype(data.dtype)

    return framerate, dataNew

---
### 正弦波

データを準備します．

In [ ]:
framerate = 8000       # 標本化周波数
vmax = 2**(15-2)       # 16bit量子化で最大振幅の 1/4
duration = 2*framerate # 2[s]

t = np.arange(duration)

freq1 = [200, 400, 600]
freq2 = [1200, 400, 800]

# signal1
signal1 = vmax * np.sin((2*np.pi*t/(framerate/freq1[0]))) \
 + vmax/2 * np.sin((2*np.pi*t/(framerate/freq1[1]))) \
 + vmax/4 * np.sin((2*np.pi*t/(framerate/freq1[2]))) \
 + vmax/2 * np.random.randn(len(t)) # ランダムノイズ

# signal2
signal2 = vmax * np.sin((2*np.pi*t/(framerate/freq2[0]))) \
 + vmax/2 * np.sin((2*np.pi*t/(framerate/freq2[1]))) \
 + vmax/4 * np.sin((2*np.pi*t/(framerate/freq2[2]))) \
 + vmax/2 * np.random.randn(len(t)) # ランダムノイズ

# グラフに描く
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(signal1, '-o', label='signal1')
ax.plot(signal2, '-o', label='signal2')
ax.axhline(0, color='gray')
ax.set_xlim(0, 100)
ax.legend()
plt.show()

`signal1` と `signal2` という2つの音のデータを作りました．どちらも，周波数の異なる3つの正弦波を足し合わせたものにノイズを乗せてあります．上のグラフからそれぞれがどんな音か想像する...のは難しいので，WAVE 形式のファイルとして出力して手元のPCにダウンロードし，音を鳴らしてみましょう．

1. 次のコードセルに記されたコメントに従ってコードを修正して実行
1. 手元に `signal1.wav` と `signal2.wav` というファイルがダウンロードされる．初めて実行すると，「複数ファイルの一括ダウンロードの許可」を求めるポップアップウィンドウが現れるかもしれません．その場合，許可するとダウンロードが始まります．
1. 手元のPCでダウンロードされたファイルを再生．WAVE 形式のファイルは，Windows でも macOS でも，OS標準の音楽プレイヤーで再生できるはずです．

In [ ]:
from google.colab import files

if 1 == 0: # ← の 0 を 1 に修正
    write('signal1.wav', framerate, signal1.astype('int16'))
    write('signal2.wav', framerate, signal2.astype('int16'))
    files.download('signal1.wav')
    files.download('signal2.wav')

次のセルを実行すると，`signal1` と `signal2` に DFT を適用してそれぞれのフーリエ係数を求め，振幅スペクトルを描きます．

In [ ]:
N = len(signal1)
freq = fftfreq(N, d=1/framerate)

# signal1 の振幅スペクトル
c_dft = fft(signal1)
amp1 = np.abs(c_dft)/N # 絶対値

# signal2 の振幅スペクトル
c_dft = fft(signal2)
amp2 = np.abs(c_dft)/N # 絶対値

fig, ax = plt.subplots(2, 3, figsize=(16, 8))

ax[0, 0].stem(amp1, markerfmt=' ', label='signal1 |Ck|')
ax[0, 0].set_xlabel('k')
ax[0, 0].legend()

ax[0, 1].stem(amp1, markerfmt=' ', label='signal1 |Ck|')
ax[0, 1].set_xlim(0, 3000)
ax[0, 1].set_xticks(np.arange(0, 3000, 400))
ax[0, 1].set_xlabel('k')
ax[0, 1].legend()

ax[0, 2].stem(freq, amp1, markerfmt=' ', label='signal1 |Ck|')
ax[0, 2].set_xlim(0, 1500)
ax[0, 2].set_xticks(np.arange(0, 1500, 200))
ax[0, 2].set_xlabel('Hz')
ax[0, 2].legend()

ax[1, 0].stem(amp2, markerfmt=' ', label='signal2 |Ck|')
ax[1, 0].set_xlabel('k')
ax[1, 0].legend()

ax[1, 1].stem(amp2, markerfmt=' ', label='signal2 |Ck|')
ax[1, 1].set_xlim(0, 3000)
ax[1, 1].set_xticks(np.arange(0, 3000, 400))
ax[1, 1].set_xlabel('k')
ax[1, 1].legend()

ax[1, 2].stem(freq, amp2, markerfmt=' ', label='signal2 |Ck|')
ax[1, 2].set_xlim(0, 1500)
ax[1, 2].set_xticks(np.arange(0, 1500, 200))
ax[1, 2].set_xlabel('Hz')
ax[1, 2].legend()

plt.show()

上段が `signal1` の振幅スペクトル，下段が `signal2` の振幅スペクトルです．
左は，横軸 $k$，縦軸 $|c_k|$ で横軸の範囲を $[0, N-1]$ として描いたもの，真ん中は，横軸の範囲を狭めたものです．
右の図では，横軸の範囲は真ん中と同じですが，横軸の単位を周波数 [Hz] として描いています．

`signal1` と `signal2` のどちらの音も，周波数の異なる3つの正弦波を足し合わせたものにノイズを乗せて作ってあります．元のデータの波形を眺めたり音を聞いたりしただけではどんな成分が含まれているのか分かりづらいですが，振幅スペクトルを見ると一目瞭然となっています．

#### 問題1

次のことをやろう／考えよう．

(1) `signal1` と `signal2` は，どちらの方が高い音に聞こえますか？

(2) それぞれの音に含まれる成分のうち，主要な（振幅が大きい）成分は，周波数が何Hzのものですか？ それぞれの音で3つずつ答えなさい．

(3) 振幅スペクトルによると，`signal1` と `signal2` ではどちらの方が高い周波数の成分を含んでいますか？ （(2)で求めた主要な成分に注目して考えよう）




---
### 猫の鳴き声

別の音のデータで実験しましょう．



WAVE ファイルを入手します．

In [ ]:
# WAVE ファイルを入手
#
### こちらのサイトの素材を利用させてもらってます http://www.ne.jp/asahi/music/myuu/wave/wave.htm

! wget -nc http://www.ne.jp/asahi/music/myuu/wave/cat1.wav
fnCat = 'cat1.wav'

import os

if not os.path.exists(fnCat):
    print(f'{fnCat}のダウンロードがうまくできていないようです')

次のセルを実行すると，`cat1` と `cat2` という二つの配列を作ります．

In [ ]:
# cat1
fr_cat1, cat1 = readWAVE('cat1.wav')
N_cat1 = len(cat1)
print(fr_cat1, N_cat1)

# cat2
cat2 = np.copy(cat1)
N_cat2 = len(cat2)
fr_cat2 = fr_cat1//2
print(fr_cat2, N_cat2)
write('cat2.wav', fr_cat2, cat2.astype('int16'))

`WavFileWarning` という警告が出るかもしれませんが，気にしないでokです．


次のセルをコメントに従って修正してから実行し，それぞれのWAVEファイルをダウンロードして音を聞いてみてください．

In [ ]:
from google.colab import files

if 1 == 0: # ← の 0 を 1 に修正
    files.download('cat1.wav')
    files.download('cat2.wav')

#### 問題2

上記の `cat1` と `cat2` のデータに DFT を適用して振幅スペクトルを描くと，下図のようになりました．音を聞いた結果から，どちらが `cat1` でどちらが `cat2` かを考えなさい．


<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/ex13catspec.png">

---
### 楽器が出す音の性質と音の高さの知覚

ギターのような弦楽器では，両端が固定された（指で押さえることで固定される位置が変わったりもしますね）弦が振動しそれが空気を振動させることで音が生み出されます．このとき，弦の振動は，下図に示すようなパターンを示します．左は両端が固定された弦の振動としては最も波長が長いもの，真ん中は波長がその $\frac{1}{2}$，右は波長 $\frac{1}{3}$ の振動を表しています．これら以外に波長 $\frac{1}{4}, \frac{1}{5}, \ldots$ の振動も現れます．
実際の弦の振動はこれらを合成したものとなりますので，そこから生まれる空気の振動もまた，ある波長の波と，その $\frac{1}{自然数}$ 倍の波長の波が混ざったものとなります．
したがって，弦楽器から生み出される音には，最も波長の長い振動に対応した最も低い周波数（これを基本周波数といいます）の成分と，その自然数倍の周波数の成分（倍音）が含まれます．ちなみに，管楽器の場合は少し様子が異なりますが，その音はやはり基本周波数の成分とその倍音から成ります．

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 3))

t = np.linspace(0, np.pi, num=100)

# 基本波長
ax[0].plot(t, np.sin(t), '-', color='red')
ax[0].set_ylim(-1.5, 1.5)
ax[0].plot(t, -np.sin(t), '--', color='red')
ax[0].axhline(0, color='gray')
ax[0].xaxis.set_visible(False)
ax[0].yaxis.set_visible(False)

# 波長 1/2 = 周波数 2 倍
ax[1].plot(t, np.sin(2*t), '-', color='red')
ax[1].set_ylim(-1.5, 1.5)
ax[1].plot(t, -np.sin(2*t), '--', color='red')
ax[1].axhline(0, color='gray')
ax[1].xaxis.set_visible(False)
ax[1].yaxis.set_visible(False)

# 波長 1/3 = 周波数 3 倍
ax[2].plot(t, np.sin(3*t), '-', color='red')
ax[2].set_ylim(-1.5, 1.5)
ax[2].plot(t, -np.sin(3*t), '--', color='red')
ax[2].axhline(0, color='gray')
ax[2].xaxis.set_visible(False)
ax[2].yaxis.set_visible(False)

一方，人間の聴覚系が知覚する音の高さは基本周波数と強く関連しており，基本的には，基本周波数の高い音ほど高い音に聞こえます．楽器や演奏の仕方が異なると，同じ音の高さでも音色が異なって聞こえますが，これは，基本周波数が同じで，倍音成分の含まれ方が異なるためです．基本周波数の成分がほとんど含まれず周波数2倍以上の倍音成分しか含まれないような音でも，基本周波数に対応した高さの音として知覚されます．

---
### ギターの音

ギターの音を録音したデータから，何の音が鳴っているのかを当てよう．


WAVE ファイルを入手します．

In [ ]:
# WAVE ファイルを入手
#
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar1-C.wav
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar2-E.wav
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar3-G.wav
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar4-ChordC.wav
fnList = ['Sound-Guitar1-C.wav', 'Sound-Guitar2-E.wav', 'Sound-Guitar3-G.wav', 'Sound-Guitar4-ChordC.wav']

import os

for fn in fnList:
    if not os.path.exists(fn):
        print(f'{fn}のダウンロードがうまくできていないようです')

4つのWAVEファイルをダウンロードして聞いてみましょう．
音感の優れたひとは聞いただけで何の音かわかるでしょうが，わからないふりをして先へ進みましょう．
ちなみに，これらの音は，本物のギターを鳴らして録音したものではなく，コンピュータで作り出した音です．ギターの音っぽい倍音成分のパターンを知っていれば，それっぽい波形を合成できる，というわけです．

In [ ]:
from google.colab import files

if 1 == 0: # ← の 0 を 1 に修正
    for fn in fnList:
        files.download(fn)

4つのWAVEファイルのデータを読み込んで，np.array のリストに格納します．`guitar[0]`, `guitar[1]`, `guitar[2]`, `guitar[3]` のそれぞれが音のデータを格納した1次元配列です．

In [ ]:
guitar = []

for i, fn in enumerate(fnList):
    print(f'##### guitar[{i}] #####')
    fr, dat = readWAVE(fn)
    dat = dat.astype(float)
    dat -= np.mean(dat)
    guitar.append(dat)
    if i == 0:
        N_guitar = len(dat)
        framerate = fr
    else:
        assert len(dat) == N_guitar and fr == framerate
    print()

それぞれの音の信号の波形を眺めてみましょう．

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(16, 10))

t = np.linspace(0, N_guitar/framerate, num=N_guitar, endpoint=False)

for i in range(len(guitar)):
    ax[i, 0].plot(t, guitar[i], label=f'guitar[{i}]')
    ax[i, 0].set_ylim(-150, 150)
    ax[i, 0].legend()
    ax[i, 1].plot(t, guitar[i], 'o', markersize=4, label=f'guitar[{i}]')
    ax[i, 1].axhline(0, color='gray')
    ax[i, 1].set_xlim(0.3, 0.305)
    ax[i, 1].set_ylim(-150, 150)
    ax[i, 1].legend()

plt.show()

それぞれの図は，横軸を時間（単位は[s]（秒））として音の信号の波形を描いたものです．4つとも2秒強の音です．右の図は，時刻 $0.3$[s] から $0.305[s]$ までの $0.005$ [s] 間のデータ点をプロットしたものです．
これらを見ただけでは，どれがどんな音なのかまるで想像がつきません．

では，これらのデータに DFT を適用してフーリエ係数を求め，振幅スペクトルを描いてみましょう．

In [ ]:
# amp[i] が guitar[i] の振幅スペクトル
amp = []
for i in range(len(guitar)):
    coef = fft(guitar[i]) # DFT
    amp.append(np.abs(coef)/N_guitar) # 振幅スペクトル
freq = fftfreq(N_guitar, d=1/framerate)

# 振幅スペクトルを描く
fig, ax = plt.subplots(4, 3, figsize=(16, 10))

for i in range(len(guitar)):
    ax[i, 0].stem(amp[i], markerfmt=' ', label=f'guitar[{i}] |Ck|')
    ax[i, 0].set_xlabel('k')
    ax[i, 0].legend()
    ax[i, 1].stem(amp[i], markerfmt=' ', label=f'guitar[{i}] |Ck|')
    ax[i, 1].set_xlim(0, 3000)
    ax[i, 1].set_xlabel('k')
    ax[i, 1].legend()
    ax[i, 2].stem(freq, amp[i], markerfmt=' ', label=f'guitar[{i}] |Ck|')
    ax[i, 2].set_xlim(0, 3000*framerate/N_guitar)
    ax[i, 2].set_xlabel('Hz')
    ax[i, 2].legend()

plt.show()

図の見方については，この notebook の上の方で同じようなことをやっているので，説明を省略します．右端の列だけ横軸の単位が違っていることに注意してください．いずれの振幅スペクトルも「弦楽器が出す音の性質」で説明したことを反映したものとなっているのが分かります．

次のセルを実行すると，上の右端の列のどれか一つの振幅スペクトルを拡大表示させることができます．コメントにしたがって `i, xmin, xmax, step` の値を適当に定めて実行してみよう．

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

# guitar[i] の i (i = 0, 1, 2, 3)
i = 0

# [xmin, xmax] が横軸の範囲，step が目盛の間隔
xmin, xmax, step = 0, 600, 20

ax.stem(freq, amp[i], markerfmt=' ', label=f'guitar[{i}] |Ck|')
ax.set_xlim(xmin, xmax) # 横軸の範囲
ax.set_ylim(0, 6) # 縦軸の範囲
ax.set_xticks(np.arange(xmin, xmax, step))
ax.set_xlabel('Hz')
ax.legend()

plt.setp(ax.get_xticklabels(), rotation=60, ha='right')
plt.show()

#### 問題3

次のことを考えて／調べてメモしておこう：

(1) `guitar[0]`, `guitar[1]`, `guitar[2]` の振幅スペクトルを観察して，それぞれの音の基本周波数を推測しよう．下の表を用いて，それが何の音であるかを当てよう．

(2) `guitar[3]` の振幅スペクトルと `guitar[0]`, `guitar[1]`, `guitar[2]` の振幅スペクトルの間にはどんな関係があるか考えよう．

|基本周波数[Hz]|音名|ハ長調での...|
|---:|:---:|:---:|
|261.6|C4|ド|
|293.7|D4|レ|
|329.6|E4|ミ|
|349.2|F4|ファ|
|392.0|G4|ソ|
|440|A4|ラ|
|493.9|B4|シ|

<span style="font-size: 75%">
※注: ここでは，A4 の基本周波数を 440[Hz]として十二平均律を用いたときの1オクターブの範囲の音の基本周波数の値を示しています．
</span>

